# Assignment - Implementing graph algorithms with LLMs

Welcome to the first assignment of Course 1 in the Generative AI for Software Development Specialization!

## 1 - Introduction

In this assignment, you'll start by working with a basic `Graph` class that comes with simple methods. Then, you'll move on to another class named `Graph_Advanced`. Here, you'll be tasked with implementing four methods to tackle four different problems.

It's important to note that the Graph class cannot be modified in any way. However, you're free to add as many methods as you need to the `Graph_Advanced` class to address the challenges you'll encounter.

We expect you to leverage an LLM (Large Language Model) to aid in developing the algorithms. These algorithms should not only aim for solutions that are close to optimal but also adhere to a specific time execution limit. We provide an LLM with GPT-3.5 for you, the link is [here](https://www.coursera.org/learn/introduction-to-generative-ai-for-software-development/ungradedLab/Vuqvf/gpt-3-5-environment), but feel free to use the LLM you want!

[**LINK FOR GPT-3.5**](https://www.coursera.org/learn/introduction-to-generative-ai-for-software-development/ungradedLab/Vuqvf/gpt-3-5-environment)

Don't worry if things seem a bit unclear at the moment! You'll receive guidance throughout the assignment.

We expect you to use LLMs in the following manner:

- Begin by clearly defining the problem you are trying to solve or the concept you wish to understand better. Present this to the LLM in a concise manner.
- Engage with the LLM as if it were a knowledgeable peer. Ask questions, seek clarifications, and propose hypotheses for the LLM to comment on.
- Provide the LLM the Graph class given to you and ask it to explain how it works.
- After receiving input from the LLM, critically evaluate the information and decide how best to apply it to your work. Not all suggestions from the LLM may be directly applicable or correct; part of your task is to discern the most valuable advice.


#### TIPS FOR SUCCESSFUL GRADING OF YOUR ASSIGNMENT:

- All cells are frozen except for the ones where you need to submit your solutions.

- You can add new cells to experiment but these will be omitted by the grader, so don't rely on newly created cells to host your solution code, use the provided places for this.

- To submit your notebook, save it and then click on the blue submit button at the beginning of the page.

### 1.1 Importing necessary libraries

Let's begin by importing the necessary libraries. **It's crucial to remember that you should not use any libraries beyond those imported for this assignment. Please communicate this constraint clearly to the LLM. Importing additional libraries could disrupt the autograder, preventing you from receiving a grade.**

In [1]:
import random
import heapq
import os
import numpy as np
import itertools

### 1.2 Importing unittests library

This library includes basic unittests to evaluate your solution. If you don't pass some of these unittests, it's probable that your solution will also fail upon submission. However, passing all the unittests doesn't guarantee success in this assignment, as the autograder will subject your solutions to more extensive testing. To successfully pass this assignment, your solution needs to meet the requirements in more than 85% of the test cases. If your solution fails, you will receive up to three examples highlighting where your algorithm fell short, along with instructions on how to replicate the graphs for debugging purposes.

In [2]:
import unittests

Here is the Graph class you'll be working with. Please take a moment to familiarize yourself with it and its methods. Feel free to ask an LLM to help explain how the class functions!

In [3]:
class Graph:
    def __init__(self, directed=False):
        """
        Initialize the Graph.

        Parameters:
        - directed (bool): Specifies whether the graph is directed. Default is False (undirected).

        Attributes:
        - graph (dict): A dictionary to store vertices and their adjacent vertices (with weights).
        - directed (bool): Indicates whether the graph is directed.
        """
        self.graph = {}
        self.directed = directed
    
    def add_vertex(self, vertex):
        """
        Add a vertex to the graph.

        Parameters:
        - vertex: The vertex to add. It must be hashable.

        Ensures that each vertex is represented in the graph dictionary as a key with an empty dictionary as its value.
        """
        if not isinstance(vertex, (int, str, tuple)):
            raise ValueError("Vertex must be a hashable type.")
        if vertex not in self.graph:
            self.graph[vertex] = {}
    
    def add_edge(self, src, dest, weight):
        """
        Add a weighted edge from src to dest. If the graph is undirected, also add from dest to src.

        Parameters:
        - src: The source vertex.
        - dest: The destination vertex.
        - weight: The weight of the edge.
        
        Prevents adding duplicate edges and ensures both vertices exist.
        """
        if src not in self.graph or dest not in self.graph:
            raise KeyError("Both vertices must exist in the graph.")
        if dest not in self.graph[src]:  # Check to prevent duplicate edges
            self.graph[src][dest] = weight
        if not self.directed and src not in self.graph[dest]:
            self.graph[dest][src] = weight
    
    def remove_edge(self, src, dest):
        """
        Remove an edge from src to dest. If the graph is undirected, also remove from dest to src.

        Parameters:
        - src: The source vertex.
        - dest: The destination vertex.
        """
        if src in self.graph and dest in self.graph[src]:
            del self.graph[src][dest]
        if not self.directed:
            if dest in self.graph and src in self.graph[dest]:
                del self.graph[dest][src]
    
    def remove_vertex(self, vertex):
        """
        Remove a vertex and all edges connected to it.

        Parameters:
        - vertex: The vertex to be removed.
        """
        if vertex in self.graph:
            # Remove any edges from other vertices to this one
            for adj in list(self.graph):
                if vertex in self.graph[adj]:
                    del self.graph[adj][vertex]
            # Remove the vertex entry itself
            del self.graph[vertex]
    
    def get_adjacent_vertices(self, vertex):
        """
        Get a list of vertices adjacent to the specified vertex.

        Parameters:
        - vertex: The vertex whose neighbors are to be retrieved.

        Returns:
        - List of adjacent vertices. Returns an empty list if vertex is not found.
        """
        return list(self.graph.get(vertex, {}).keys())

    def _get_edge_weight(self, src, dest):
        """
        Get the weight of the edge from src to dest.

        Parameters:
        - src: The source vertex.
        - dest: The destination vertex.

        Returns:
        - The weight of the edge. If the edge does not exist, returns infinity.
        """
        return self.graph[src].get(dest, float('inf'))
    
    def __str__(self):
        """
        Provide a string representation of the graph's adjacency list for easy printing and debugging.

        Returns:
        - A string representation of the graph dictionary.
        """
        return str(self.graph)

In [4]:
g = Graph(directed=False)
g.add_vertex("A")
g.add_vertex("B")
g.add_vertex("C")  # Add vertex "C"
g.add_edge("A", "B", 5)
g.add_edge("B", "C", 3)
print(g)

{'A': {'B': 5}, 'B': {'A': 5, 'C': 3}, 'C': {'B': 3}}


You are required to modify a new class, which inherits from the Graph class, by filling in the blank methods listed below. It's entirely acceptable, and often necessary, to add auxiliary (helper) methods to address each problem effectively. 

**However, the primary function tasked with solving each problem must retain the names provided below and return the output in the specified order: first the distance, followed by the path. Variable names in the return call are just examples. You can change them, as long as the first argument is the distance and the second is the path, given by a list of node numbers.**

## Example of an expected function output

```Python
graph = Graph_Advanced() # Assume the graph is already built with nodes and edges

print(graph.shortest_path(5, 260)) # Find the shortest path between node 5 and 260.

310, [5,17, 80, 71, 190, 260]

```

## 2 - The Graph_Advanced Class

Below is the Graph_Advanced class, which is a subclass of the Graph class. You are tasked with implementing 4 methods, each corresponding to an exercise in this assignment. It's crucial to remember that you **must not** alter the names of these functions, and the output format must be (distance, path) as specified. You **can** introduce auxiliary methods **within the Graph_Advanced class** to aid in solving any of the exercises, but the final solution **must** be delivered through the functions listed below.

### 2.1 How to Complete The Exercises with LLM Assistance

1. **Problem Understanding with LLM**: Discuss graph theory problems like the Travelling Salesman Problem (TSP) and shortest path finding with the LLM. Use it to clarify concepts and get examples, enhancing your foundational knowledge.

2. **Graph Class Analysis with LLM**: Analyze the provided Graph class functions and structure by consulting the LLM. Its explanations will help you understand how to effectively utilize and extend the class.

3. **Method Implementation Guidance**: For coding tasks (`shortest_path`, `tsp_small_graph`, `tsp_large_graph`, `tsp_medium_graph`), brainstorm with the LLM. Share your logic and seek optimization tips to refine your solutions. You can even ask the LLM to build the code for you, with any specification you want.

4. **Developing Helper Methods**: Before adding helper methods to the `Graph_Advanced` class, discuss your ideas with the LLM for feedback on implementation strategies, ensuring they align with the class's integrity.

5. **Testing Solutions with LLM**: Use the LLM to strategize and review your testing approach, especially for tests that yield unexpected results. It can offer debugging and optimization advice to improve your solutions.

Keep the interaction with the LLM continuous, treating it as a mentor for advice, debugging, and review throughout the exercise process. This approach will not only aid in task completion but also in skill enhancement.

In [16]:
import heapq
from joblib import Parallel, delayed
import random

class Graph_Advanced(Graph):
    
    # Exercise 1: The Shortest Path Challenge
    def shortest_path(self, start: int, end: int) -> tuple[float, list[int]]:
        """
        Calculate the shortest path from a starting node to an ending node in a sparse graph
        with 10,000 nodes. Must execute in less than 0.5 seconds.

        Uses Dijkstra's algorithm with a priority queue (min-heap).
        """
        if start == end:
            return 0, [start]  # Early exit for trivial case
        
        if start not in self.graph or end not in self.graph:
            raise ValueError("Start or end vertex not found in graph.")
        
        # Priority queue for exploring nodes based on shortest known distance
        pq = [(0, start)]  # (distance, vertex)
        distances = {vertex: float('inf') for vertex in self.graph}
        distances[start] = 0
        previous = {vertex: None for vertex in self.graph}

        while pq:
            current_distance, current_vertex = heapq.heappop(pq)

            if current_vertex == end:
                break  # Early exit if we reach the destination

            if current_distance > distances[current_vertex]:
                continue

            for neighbor, weight in self.graph[current_vertex].items():
                distance = current_distance + weight

                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous[neighbor] = current_vertex
                    heapq.heappush(pq, (distance, neighbor))

        # Reconstruct path
        path = []
        vertex = end
        while vertex is not None:
            path.insert(0, vertex)
            vertex = previous[vertex]
        
        if distances[end] == float('inf'):
            return float('inf'), []  # No path found
        return distances[end], path

    # Exercise 2: Solving the TSP for Small Graphs    
    def tsp_small_graph(self, start_vertex=0) -> tuple[float, list[int]]:
        """
        Solve the Traveling Salesman Problem for small (~10 node) graphs. The algorithm must find 
        the optimal solution starting from node 0 and complete in under 1 second.

        This version uses the Held-Karp algorithm (dynamic programming with bitmasking).
        """
        vertices = list(self.graph.keys())
        n = len(vertices)

        if n > 10:
            raise ValueError("Graph size exceeds limit for small graphs (10 nodes).")
        
        if start_vertex not in self.graph:
            raise ValueError("Start vertex not found in the graph.")
        
        index_map = {v: i for i, v in enumerate(vertices)}
        
        # Precompute edge weights for faster lookup, excluding self-loops
        edge_weights = {(i, j): self._get_edge_weight(vertices[i], vertices[j]) 
                        for i in range(n) for j in range(n) if i != j}

        # DP table and parent table for path reconstruction
        dp = [[float('inf')] * n for _ in range(1 << n)]
        dp[1 << index_map[start_vertex]][index_map[start_vertex]] = 0
        parent = [[-1] * n for _ in range(1 << n)]

        # Dynamic programming with bitmasking
        for mask in range(1 << n):
            for u in range(n):
                if mask & (1 << u):  # Check if u is in the current set
                    for v in range(n):
                        if not (mask & (1 << v)):  # Check if v is not in the current set
                            new_mask = mask | (1 << v)
                            new_dist = dp[mask][u] + edge_weights.get((u, v), float('inf'))
                            if new_dist < dp[new_mask][v]:
                                dp[new_mask][v] = new_dist
                                parent[new_mask][v] = u

        # Find the optimal path, ensuring valid edges between distinct vertices
        all_visited = (1 << n) - 1
        final_dist = min(dp[all_visited][v] + edge_weights.get((v, index_map[start_vertex]), float('inf'))
                         for v in range(n) if v != index_map[start_vertex])
        
        # Path reconstruction
        path = []
        mask = all_visited
        end_vertex = min(range(n), key=lambda v: dp[all_visited][v] + edge_weights.get((v, index_map[start_vertex]), float('inf')))
        while end_vertex != -1:
            path.append(vertices[end_vertex])
            new_end = parent[mask][end_vertex]
            mask ^= (1 << end_vertex)
            end_vertex = new_end
        
        path.append(start_vertex)
        path.reverse()

        return final_dist, path

    # Exercise 3: Tackling the TSP for Large Graphs
    def tsp_large_graph(self, start_vertex: int) -> tuple[float, list[int]]:
        """
        Solve the Traveling Salesman Problem for large (~1000 node) graphs. The goal is to find a
        good approximation using a nearest neighbor heuristic, executing in under 0.5 seconds.
        """
        if start_vertex not in self.graph:
            raise ValueError("Start vertex not found in the graph.")

        path = [start_vertex]
        total_distance = 0
        unvisited = set(self.graph.keys())
        unvisited.remove(start_vertex)
        current_vertex = start_vertex

        while unvisited:
            next_vertex = min(unvisited, key=lambda vertex: self._get_edge_weight(current_vertex, vertex))
            total_distance += self._get_edge_weight(current_vertex, next_vertex)
            path.append(next_vertex)
            unvisited.remove(next_vertex)
            current_vertex = next_vertex

        # Return to the starting vertex to complete the tour
        total_distance += self._get_edge_weight(current_vertex, start_vertex)
        path.append(start_vertex)

        return total_distance, path

    # Exercise 4: The TSP Medium Graphs Challenge    
    def tsp_medium_graph(self, start_vertex: int) -> tuple[float, list[int]]:
        """
        Solve the Traveling Salesman Problem for medium (~300 node) graphs. The goal is to improve
        upon the nearest neighbor solution using 2-opt optimization, executing in less than 1.3 seconds.
        This version runs the nearest neighbor search from multiple start points in parallel.
        """
        # Precompute edge weights for faster lookup
        vertices = list(self.graph.keys())
        n = len(vertices)
        edge_weights = {(i, j): self._get_edge_weight(vertices[i], vertices[j]) for i in range(n) for j in range(n) if i != j}

        # Nearest neighbor heuristic for a single start point
        def nearest_neighbor(start: int):
            path = [start]
            unvisited = set(vertices)
            unvisited.remove(start)
            total_distance = 0
            current = start

            while unvisited:
                next_vertex = min(unvisited, key=lambda vertex: edge_weights.get((current, vertex), float('inf')))
                total_distance += edge_weights.get((current, next_vertex), float('inf'))
                path.append(next_vertex)
                unvisited.remove(next_vertex)
                current = next_vertex

            total_distance += edge_weights.get((current, start), float('inf'))  # Return to start
            path.append(start)
            return total_distance, path

        # Run nearest neighbor from 3 different start points in parallel
        start_points = random.sample(vertices, 3)  # Try 3 different starting points to save time
        results = Parallel(n_jobs=-1)(delayed(nearest_neighbor)(start) for start in start_points)

        # Select the best nearest neighbor solution
        best_distance, best_path = min(results, key=lambda x: x[0])

        # Apply 2-opt optimization to improve the solution
        def two_opt(route: list[int], max_iterations: int = 50) -> list[int]:
            best = route
            iterations = 0
            improved = True

            while improved and iterations < max_iterations:
                iterations += 1
                improved = False
                # Batch random edge swaps rather than checking all pairs
                for _ in range(5):  # Limit to 5 random swaps per iteration
                    i, j = sorted(random.sample(range(1, len(route) - 1), 2))
                    if j - i == 1: continue  # Skip adjacent nodes
                    new_route = route[:i] + route[i:j][::-1] + route[j:]
                    if self._tour_cost(new_route) < self._tour_cost(best):
                        best = new_route
                        improved = True

                if not improved:
                    break  # Early exit if no improvement
            return best

        # Perform 2-opt optimization on the best nearest neighbor result
        optimized_path = two_opt(best_path, max_iterations=50)  # Further reduced iterations for speed
        optimized_dist = self._tour_cost(optimized_path)
        return optimized_dist, optimized_path

    # Helper function to calculate the total cost of a given tour
    def _tour_cost(self, path: list[int]) -> float:
        """
        Helper function to calculate the total cost of a given tour.
        """
        total_distance = 0
        for i in range(len(path) - 1):
            total_distance += self._get_edge_weight(path[i], path[i+1])
        return total_distance



## 3 - Exercises

### 3.1 Exercises description

### Exercise 1: The Shortest Path Challenge

Develop an algorithm to find the quickest route between two points in a sparse graph containing 10,000 nodes, ensuring it executes in less than `0.5` seconds. If you're unsure about your approach or need optimization advice, the LLM is there to provide insights and strategies.

### Exercise 2: Solving the TSP for Small Graphs

Your task is to create a function that solves the Traveling Salesman Problem for small graphs of about 10 nodes, starting at node 0. The solution must be efficient, completing the task in under `1` second. The LLM can serve as a valuable resource for exploring algorithmic solutions and improving your code's efficiency.

### Exercise 3: Tackling the TSP for Large Graphs

Address the Traveling Salesman Problem for large graphs with 1,000 nodes. The goal isn't to find the perfect solution but to approach it closely within `0.5` seconds. When faced with the complexity of this challenge, the LLM can offer guidance on heuristic approaches and performance optimization.

### Exercise 4: The TSP Medium Graphs Challenge

Focus on medium-sized graphs with 300 nodes and develop an algorithm that performs better than your solution for larger graphs, with a completion time of less than `1.3` seconds. The LLM is ready to assist by sharing insights on algorithmic efficiency and helping you surpass your previous solutions.

### 3.2 Unit Testing

Unit testing functions are provided to validate your algorithms. Run these without modifications after completing each exercise. They offer immediate feedback, including failure reasons and guidance for improving your solutions. You can test solutions individually, ensuring focused and efficient debugging.

### 3.3 Generate Graph Function

The function outlined below can assist you in experimenting, debugging, and testing your code while you work on the algorithms. If your algorithm fails a test case, you will be given the call to this function with the appropriate arguments needed to replicate the graph that caused your algorithm to fail. Additionally, the reason for the failure will be provided, whether it was due to exceeding the time limit or not achieving an optimal or near-optimal distance. Feel free to ask the LLM to explain how this function works.

In [6]:
def generate_graph(nodes, edges=None, complete=False, weight_bounds=(1,600), seed=None):
    """
    Generates a graph with specified parameters, allowing for both complete and incomplete graphs.
    
    This function creates a graph with a specified number of nodes and edges, with options for creating a complete graph, and for specifying the weight bounds of the edges. It uses the Graph_Advanced class to create and manipulate the graph.

    Parameters:
    - nodes (int): The number of nodes in the graph. Must be a positive integer.
    - edges (int, optional): The number of edges to add for each node in the graph. This parameter is ignored if `complete` is set to True. Defaults to None.
    - complete (bool, optional): If set to True, generates a complete graph where every pair of distinct vertices is connected by a unique edge. Defaults to False.
    - weight_bounds (tuple, optional): A tuple specifying the lower and upper bounds (inclusive) for the random weights of the edges. Defaults to (1, 600).
    - seed (int, optional): A seed for the random number generator to ensure reproducibility. Defaults to None.

    Raises:
    - ValueError: If `edges` is not None and `complete` is set to True, since a complete graph does not require specifying the number of edges.

    Returns:
    - Graph_Advanced: An instance of the Graph_Advanced class representing the generated graph, with vertices labeled as integers starting from 0.

    Examples:
    - Generating a complete graph with 5 nodes:
        generate_graph(5, complete=True)
    
    - Generating an incomplete graph with 5 nodes and 2 edges per node:
        generate_graph(5, edges=2)
    
    Note:
    - The function assumes the existence of a Graph_Advanced class with methods for adding vertices (`add_vertex`) and edges (`add_edge`), as well as a method for getting adjacent vertices (`get_adjacent_vertices`).
    """
    random.seed(seed)
    graph = Graph_Advanced()
    if edges is not None and complete:
        raise ValueError("edges must be None if complete is set to True")
    

    for i in range(nodes):
        graph.add_vertex(i)
    if complete:
        for i in range(nodes):
            for j in range(i+1,nodes):
                weight = random.randint(weight_bounds[0], weight_bounds[1])
                graph.add_edge(i,j,weight)
    else:
        for i in range(nodes):
            for _ in range(edges):
                j = random.randint(0, nodes - 1)
                while j == i or j in graph.get_adjacent_vertices(i):  # Ensure the edge is not a loop or a duplicate
                    j = random.randint(0, nodes - 1)
                weight = random.randint(weight_bounds[0], weight_bounds[1])
                graph.add_edge(i, j, weight)
    return graph

In [7]:
# Generate a complete graph with 5 nodes
g_complete = generate_graph(5, complete=True)

# Generate an incomplete graph with 5 nodes and 2 edges per node
#g_incomplete = generate_graph(5, edges=2)

# Generate a graph with random seed for reproducibility
#g_seeded = generate_graph(300, complete=True, seed=42)


In [8]:
# Generate a graph with random seed for reproducibility
g_seeded = generate_graph(300, complete=True, seed=42)

## 4 - Playground

This section is your space to experiment with and test your methods. To measure the execution time of your code, you can use the `%%timeit` magic method. Remember, `%%timeit` should be placed at the **beginning** of a code block, even before any comments marked by `#`. It's important to note that this magic method doesn't save the output of the function it measures. You're encouraged to create as many new cells as needed for testing, but keep in mind that only the code within the `Graph_Advanced` class will be considered during grading. Here's an example of how to use `%%timeit` to assess the execution time of functions.

In [9]:
## Example function
def foo(n):
    i = 0
    for i in range(10000):
        for j in range(n):
            i += j
    return i

In [10]:
print(foo(10))

10044


In [11]:
%%timeit
foo(10)

4.64 ms ± 342 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Feel free to add as many cells as you want! You can do so by clicking in the `+` button in the left upper corner of this notebook.

## 5 - Test Your Solutions

After implementing your methods in the previous class, you can run the code blocks below to test each of your developed methods! If you see the message "All tests passed!" for every unittest, your assignment is likely ready for submission. If not, you'll be able to see which cases your code didn't pass and understand why. **Remember, the unittests here cover only a few example cases due to the constraints of this environment. The grading process will include more extensive testing. It's possible to pass all the tests here but still fail some during the autograding.** However, if that happens, don't worry—you will receive feedback on why you failed, which will allow you to adjust your functions and resubmit the assignment. You're welcome to submit your assignment multiple times as needed.

**IMPORTANT NOTE: Please ensure you shut down any other live kernels (if you have another notebook open within this environment), as they may affect the execution time of your functions.**

### 5.1 Test Exercise 1 (method Graph_Advanced.shortest_path)

Run the code below to test the `shortest_path` method on sparsely connected graphs with 10,000 nodes. The requirements for passing this exercise are:

- The algorithm must complete its run in under `0.5` second for each graph.
- It must accurately find the shortest path.

In [12]:
unittests.test_shortest_path(Graph_Advanced)

 All tests passed!


### 5.2 Test Exercise 2 (method Graph_Advanced.tsp_small_graph)

Run the code below to test the `tsp_small_graph` on complete (fully connected) graphs with 10 nodes. The requirements for passing this exercise are:

- The algorithm must complete its run in under `1` second for each graph.
- It must fund the optimal solution, starting at node 0.

In [13]:
unittests.test_tsp_small_graph(Graph_Advanced)

 All tests passed!


### 5.3 Test Exercise 3 (method Graph_Advanced.tsp_large_graph)

Run the code below to test the `tsp_large_graph` on complete (fully connected) graphs with 1000 nodes. The requirements for passing this exercise are:

- The algorithm must complete its run in under `0.5` second for each graph.
- It must find the good solution (less than a specified value, depending on the graph). 

In [14]:
unittests.test_tsp_large_graph(Graph_Advanced)

 All tests passed!


### 5.4 Test Exercise 4 (method Graph_Advanced.tsp_medium_graph)

Run the code below to test the `tsp_medium_graph` on complete (fully connected) graphs with 300 nodes. The requirements for passing this exercise are:

- The algorithm must complete its run in under `1.3` seconds for each graph.
- It must find the good solution (less than a specified value, depending on the graph). 

In [20]:
# Assuming the Graph_Advanced class is defined and a graph has been created
graph = Graph_Advanced()

# Add vertices and edges to form a complete graph
for i in range(300):
    graph.add_vertex(i)

# Example of adding edges with random weights for 300 nodes
import random
for i in range(300):
    for j in range(i + 1, 300):
        weight = random.randint(1, 100)
        graph.add_edge(i, j, weight)

# Run the tsp_medium_graph method
dist, path = graph.tsp_medium_graph(0)
print("TSP Medium Graph - Distance:", dist, "Path:", path)

TSP Medium Graph - Distance: 688 Path: [210, 13, 171, 109, 69, 31, 68, 147, 129, 151, 10, 106, 130, 17, 2, 72, 6, 61, 19, 28, 76, 167, 45, 53, 12, 240, 32, 36, 0, 181, 149, 164, 152, 66, 235, 259, 23, 239, 21, 124, 237, 38, 4, 5, 117, 11, 95, 236, 27, 79, 85, 25, 94, 150, 22, 70, 89, 116, 229, 77, 90, 184, 126, 141, 288, 41, 78, 199, 132, 221, 63, 119, 7, 156, 47, 29, 113, 110, 107, 100, 200, 206, 64, 186, 97, 99, 185, 54, 242, 176, 1, 37, 96, 154, 93, 20, 142, 160, 144, 140, 60, 217, 145, 122, 48, 16, 123, 104, 57, 34, 118, 49, 42, 51, 44, 223, 238, 74, 103, 82, 203, 280, 249, 243, 289, 15, 234, 277, 56, 226, 87, 191, 177, 224, 88, 127, 299, 50, 182, 195, 55, 232, 265, 14, 190, 270, 233, 264, 98, 143, 169, 115, 172, 133, 174, 40, 168, 114, 155, 30, 165, 153, 46, 24, 205, 62, 112, 86, 120, 166, 208, 9, 292, 131, 214, 255, 175, 196, 157, 84, 170, 222, 101, 71, 230, 252, 263, 268, 134, 58, 162, 139, 18, 138, 251, 294, 209, 276, 65, 108, 33, 80, 231, 260, 269, 67, 125, 225, 188, 179, 75, 

In [21]:
unittests.test_tsp_medium_graph(Graph_Advanced)

Failed test case: Exceeded time execution limit for a tour starting in node 0. To replicate the graph, you may run generate_graph(nodes = 300, complete = True, seed = 42).
Expected: tsp_medium_graph method must run in less than 1.3 seconds
Got: Time execution exceeded 1.3 seconds

Failed test case: Exceeded time execution limit for a tour starting in node 0. To replicate the graph, you may run generate_graph(nodes = 300, complete = True, seed = 43).
Expected: tsp_medium_graph method must run in less than 1.3 seconds
Got: Time execution exceeded 1.3 seconds

Failed test case: Exceeded time execution limit for a tour starting in node 0. To replicate the graph, you may run generate_graph(nodes = 300, complete = True, seed = 44).
Expected: tsp_medium_graph method must run in less than 1.3 seconds
Got: Time execution exceeded 1.3 seconds

Failed test case: Exceeded time execution limit for a tour starting in node 0. To replicate the graph, you may run generate_graph(nodes = 300, complete = 

**Congratulations! You completed the first assignment of the Specialization!**